In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kplus_gps_1to3p5_Jul8_ntuple/*.root")

SyntaxError: invalid syntax (data.py, line 20)

In [ ]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [ ]:
hists = {}
hists["hist_pdgID"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"PDG ID", 50, 300, 350)
                            )
hists["hist_n"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"PDG ID", 4, 0, 4)
                            )

hists["hist_ke"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"Kinetic Energy [MeV]", 100, 0, 4100)
                            )
hists["hist_e"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"Energy [MeV]", 100, 0, 4100)
                            )
hists["hist_test"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"PDG ID", 400, 0, 400)
                            )
hists["hist_dau_e"] = hist.Hist("Sim Particle",
                                hist.Bin("e", r"PDG ID", 400, 0, 400)
                            )


In [ ]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    sim_particle = ldmx_events['Sim_Particle']
    sim_particle_n = ldmx_events['n']['Sim_Particle']
    sim_particle_dau1 = ldmx_events['Sim_Particle_dau1']
    sim_particle_dau2 = ldmx_events['Sim_Particle_dau2']
    
    #kplus_pdgid  = 321
    #kminus_pdgid = -321
    kplus_mass = 493.677
    
    n_mask = ((sim_particle_n)==1)
    
    decay1_mask = (flat(sim_particle_dau1.momdecay)==1)
    decay2_mask = (flat(sim_particle_dau2.momdecay)==1)
    
    print(flat(sim_particle.decay))
    
    hists["hist_pdgID"].fill(
                         e=flat(sim_particle.pdgID),
                         ) 
    hists["hist_n"].fill(
                         e=sim_particle_n,
                         )        
    hists["hist_ke"].fill(
                         e=flat(sim_particle.e)-kplus_mass,
                         ) 
    hists["hist_e"].fill(
                         e=flat(sim_particle.e),
                         ) 
    hists["hist_test"].fill(
                         e=flat(abs(sim_particle_dau2.pdgID)),
                         )

    
    #decay 1, dau1 = electron neutrino
    #decay 1, dau2 = electron neutrino and tau- ???? what
    #     decay 2 ???? what
    #decay 3, dau1 = electron neutrino
    #decay 3, dau2 = electron neutrino and tau- and pi0???
    #     decay 4 ???? what
    #decay 7, dau1 = pi and e-
    #decay 7, dau2 = pi0
    #decay 8, dau1 = 

In [ ]:
nchunk = 0
for chunk in ldmx_dict:
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

In [ ]:
print(hists["hist_pdgID"])
fig, ax = plt.subplots()
hist.plot1d(hists["hist_pdgID"],ax=ax,clear=False);
leg = ax.legend([r'Gen $K^+$ Particle'])

fig, ax = plt.subplots()
hist.plot1d(hists["hist_e"],ax=ax,clear=False);
leg = ax.legend([r'Gen $K^+$ Particle'])


fig, ax = plt.subplots()
hist.plot1d(hists["hist_ke"],ax=ax,clear=False);
leg = ax.legend([r'Gen $K^+$ Particle'])





fig, ax = plt.subplots()
hist.plot1d(hists["hist_test"],ax=ax,clear=False);